# **Esercitazione 3 - Classificatori: Regressione Logistica e SVM**

In questa esercitazione applicheremo quanto appreso sui classificatori. Nello specifico utilizzeremo:

* **Regressione Logistica:** Un modello lineare che utilizza la funzione sigmoide per predire le probabilità delle classi.

* **Support Vector Machines (SVM):** Efficace sia per problemi lineari che non lineari utilizzando il kernel trick.

### **Dataset MNIST-784**

Il dataset di riferimento sarà `MNIST-784`, già visto in precedenza. Il dataset contiene immagini di 10 classi (da 0 a 9). Per comodità utilizzeremo soltanto 2 classi inizialmente, per rendere la classificazione binaria. Nello specifico utilizzeremo soltanto le immagini che hanno come etichetta `3` e `8`. 

Il codice seguente esegui l' import delle librerie necessarie e la selezione delle etichette che ci interessano. Le etichette vengono anche rimpiazzate con `1` e `0`, emulando il caso di classificazione binaria.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

In [30]:
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = mnist.data, mnist.target.astype(int)

indexes = (y == 3) | (y == 8)
X = X[indexes]  
y = y[indexes]

y = np.where(y == 8, 1, 0)

print(f"Features shape: {X.shape}, Labels shape: {y.shape}")

Features shape: (13966, 784), Labels shape: (13966,)


### **Divisione e standardizzazione del dataset** 

Dividiamo il dataset in `train set`, `validation set` e `test set` utilizzando le proporzioni già impostate. Successivamente applichiamo la standardizzazione utilizzando `StandardScaler`.

In [31]:
# Usare le seguenti proporzioni per il train, validation e test
train_fraction = 0.6  
validation_fraction = 0.2  
test_fraction = 0.2

train_numb = int(train_fraction * X.shape[0])
validation_numb = int(validation_fraction*X.shape[0])

X_train = X[train_numb]
y_train = y[:train_numb]
X_validation = X[train_numb:train_numb+validation_numb]
y_validation = y[train_numb:train_numb+validation_numb]
X_test = X[train_numb+validation_numb:]
y_test = y[train_numb+validation_numb:]

print("Train shape:", X_train.shape)
print("Validation shape:", X_validation.shape)
print("Test shape:", X_test.shape)

#Standardizzazione
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

KeyError: 8379

## **Esercizio 1: Implementare la Regressione Logistica**

Per implementare la regressione logistica utilizzeremo la classe `sklearn.linear_model.LogisticRegression` presente in `scikit-learn`. La documentazione è disponibile [a questo link](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

Per utilizzarla al meglio, di seguito vediamo i parametri chiave che bisogna specificare al momento della creazione dell' istanza:

* **`C`**: Inverso della forza di regolarizzazione L2 (λ). Valori più piccoli indicano una regolarizzazione più forte.
* **`solver`**: Algoritmo da utilizzare nel problema di ottimizzazione (nel nostro caso, `liblinear`).
* **`max_iter`**: Imposta il numero massimo di iterazioni affinché l'algoritmo di ottimizzazione converga e trovi i migliori parametri del modello.

### Esempio di sintassi per istanziare, addestrare e predire

```python
#Importo LogisticRegression da scikit-learn
from sklearn.linear_model import LogisticRegression

#1. Instanzio il modello di Regressione Logistica
# Durante la creazione dell' istanza imposto i parametri che desidero
model = LogisticRegression(max_iter=100, solver='liblinear',C=1.0)

#2. Train del modello utilizzando il metodo .fit()
model.fit(X_train, y_train)

#3. Calcolo delle predizioni utilizzando il metodo .predict()
predictions = model.predict(X_test)

```

### **Guida per la risoluzione**

Di seguito sono spiegati i passaggi principali per la risoluzione dell' esercizio.

1. **Creazione del modello:** Creare un' istanza della classe `LogisticRegression`, specificando i parametri presentati poco sopra. In particolare vogliamo i seguenti parametri:
    
    - `max_iter` = 100
    - `solver` = `'liblinear'`
    - `C` = 1.0

2. **Addestramento del modello:** Addestriamo il modello utilizzando il metodo `.fit()`. Il modello deve essere addestrato sui dati di train standardizzati. 

3. **Calcolo delle predizioni:** Calcoliamo le predizioni sul validation e test utilizzando il metodo `.predict()` del modello. 

4. **Valutazione delle prestazioni del modello:** Calcoliamo l' accuracy del modello. Ricordiamo che l' accuracy è data dal numero di predizioni corrette che il modello effettua rispetto al totale dei campioni. Dobbiamo valutare il modello sia sul validation set che sul test set. Infine stampare il valore di accuracy sul validation e sul test.

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# Step 1 - Creazione del modello

model_log = LogisticRegression(max_iter=100, solver='liblinear',C=1.0)

In [ ]:
# Step 2 - Addestramento del modello

model_log.fit(X_train, y_train)

In [ ]:
# Step 3 - Calcolo delle predizioni

y_predictions_validation = model_log.predict(X_validation)
y_predictions_test = model_log.predict(X_test)

In [ ]:
# Step 4 - Calcolo delle metriche di valutazione

i = 0
counter = 0
for label in y_predictions_test:
    if label == y_test[i]:
        counter = counter+1
    i = i+1
accuracy_test = counter/len(y_predictions_test)
print("Test accuracy: ", accuracy_test)

i = 0
counter = 0
for label in y_predictions_validation:
    if label == y_validation[i]:
        counter = counter+1
    i = i+1
accuracy_validation = counter/len(y_predictions_validation)
print("Validation accuracy: ", accuracy_validation)


## **Esercizio 2: Implementare Support Vector Machines (SVM)**

Per implementare le SVM utilizziamo la classe `sklearn.svm.SVC` presente in `scikit-learn`. La documentazione è disponibile [a questo link](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

Per utilizzarla al meglio, di seguito vediamo i parametri chiave che bisogna specificare al momento della creazione dell' istanza:

* **`C`**: Parametro di regolarizzazione. L'intensità della regolarizzazione è inversamente proporzionale a C.
* **`kernel`**: Specifica il tipo di kernel da utilizzare nell'algoritmo (`'linear'`, `'poly'`, `'rbf'`).

### Esempio di sintassi per istanziare, addestrare e predire

```python
#Importare la classe SVC da scikit-learn
from sklearn.svm import SVC

#1. Creare un' istanza della classe SVC
# Durante la creazione dell' istanza imposto i parametri che desidero
model = SVC(kernel='linear', C=1.0)

#2. Train del modello utilizzando il metodo .fit()
model.fit(X_train, y_train)

#3. Calcolo delle predizioni utilizzando il metodo .predict()
predictions = model.predict(X_test)

### **C** in SVM:

**C** è una penalità per i punti classificati erroneamente.

- **Small C**: Margine più ampio, tollera alcuni errori (rischio di underfitting).
- **Large C**: Minimizza gli errori, margine più stretto (rischio di overfitting).

### **Guida per la risoluzione**

Di seguito sono spiegati i passaggi principali per la risoluzione dell' esercizio.

1. **Creazione del modello:** Creare un' istanza della classe `SVC`, specificando i parametri presentati poco sopra. In particolare vogliamo i seguenti parametri:
    
    - `kernel` = `'linear'`
    - `C` = 0.01

2. **Addestramento del modello:** Addestriamo il modello utilizzando il metodo `.fit()`. Il modello deve essere addestrato sui dati di train standardizzati. 

3. **Calcolo delle predizioni:** Calcoliamo le predizioni sul validation e test utilizzando il metodo `.predict()` del modello. 

4. **Valutazione delle prestazioni del modello:** Calcoliamo l' accuracy del modello. Ricordiamo che l' accuracy è data dal numero di predizioni corrette che il modello effettua rispetto al totale dei campioni. Dobbiamo valutare il modello sia sul validation set che sul test set. Infine stampare il valore di accuracy sul validation e sul test.

In [ ]:
# Step 1 - Creazione del modello
from sklearn.svm import SVC
import numpy as np

model_SVM = SVC(kernel='linear', C=0.01)

In [ ]:
# Step 2 - Addestramento del modello

model_SVM.fit(X_train, y_train)

In [ ]:
# Step 3 - Calcolo delle predizioni

predictions = model_SVM.predict(X_test)

In [ ]:
# Step 4 - Calcolo delle metriche di valutazione

i = 0
counter = 0
for label in y_predictions_test:
    if label == y_test[i]:
        counter = counter+1
    i = i+1
accuracy_test = counter/len(y_predictions_test)
print("Test accuracy: ", accuracy_test)

i = 0
counter = 0
for label in y_predictions_validation:
    if label == y_validation[i]:
        counter = counter+1
    i = i+1
accuracy_validation = counter/len(y_predictions_validation)
print("Validation accuracy: ", accuracy_validation)

### **Esercizio 2.1: Implementare SVM con kernel trick**

Vogliamo implementare un classificatore SVM che utilizza il kernel-trick. Le linee guida sono esattamente quanto fatto prima, dobbiamo però modificare il tipo di kernel del modello. Per utilizzare il kernel trick usiamo un **kernel a base radiale (Radial Basis Function)** specificando il parametro:

- `kernel` = `'rbf'`

In [ ]:
# Step 1 - Creazione del modello
from sklearn.svm import SVC
import numpy as np

model_kernel = SVC(kernel='rbf', C=0.01)

# Step 2 - Addestramento del modello

model_kernel.fit(X_train, y_train)

# Step 3 - Calcolo delle predizioni

predictions = model_kernel.predict(X_test)

# Step 4 - Calcolo delle metriche di valutazione

i = 0
counter = 0
for label in y_predictions_test:
    if label == y_test[i]:
        counter = counter+1
    i = i+1
accuracy_test = counter/len(y_predictions_test)
print("Test accuracy: ", accuracy_test)

i = 0
counter = 0
for label in y_predictions_validation:
    if label == y_validation[i]:
        counter = counter+1
    i = i+1
accuracy_validation = counter/len(y_predictions_validation)
print("Validation accuracy: ", accuracy_validation)

## **Esercizio 3: Metriche di valutazione**

**Matrice di confusione:** 

La matrice di confusione (anche nota come `confusion matrix` ) è una tabella che riassume le prestazioni di un modello di classificazione mostrando i conteggi dei veri positivi (TP), veri negativi (TN), falsi positivi (FP) e falsi negativi (FN). In questo codice, per un problema di classificazione binaria con classi 0 e 1, la `confusion matrix` è strutturata come segue:

|                | **Predicted Class 0** | **Predicted Class 1** |
|----------------|:------------------------:|:------------------------:|
| **Actual Class 0** | TN                     | FP                     |
| **Actual Class 1** | FN                     | TP                     |

* **TN (True Negatives):** Il numero di istanze che erano effettivamente Classe 0 e sono state correttamente previste come Classe 0.

* **FP (False Positives):** Il numero di istanze che erano effettivamente Classe 0 ma sono state erroneamente previste come Classe 1.

* **FN (False Negatives):** Il numero di istanze che erano effettivamente Classe 1 ma sono state erroneamente previste come Classe 0.

* **TP (True Positives):** Il numero di istanze che erano effettivamente Classe 1 e sono state correttamente previste come Classe 1.


* **Accuracy:** Misura la correttezza complessiva del modello. È il rapporto tra le istanze correttamente classificate e il numero totale di istanze.

    $$
    \text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
    $$

* **Precision:** Misura l'accuratezza delle previsioni positive. È il rapporto tra le istanze positive correttamente previste e il numero totale di istanze previste come positive.

    $$\text{Precision} = \frac{TP}{TP + FP}$$

* **Recall (Sensitivity or True Positive Rate):** Misura la capacità del modello di trovare tutte le istanze positive. È il rapporto tra le istanze positive correttamente previste e il numero totale di vere istanze positive.

    $$\text{Recall} = \frac{TP}{TP + FN}$$

* **F1-Score:** La media armonica di precisione e richiamo. Fornisce un punteggio unico che bilancia sia la precisione che il richiamo, particolarmente utile quando c'è una distribuzione asimmetrica delle classi.

    $$\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$$

Per calcolare le metriche utilizziamo le funzioni presenti in `sklearn.metrics`:

#### `confusion_matrix`

Dati in input il target reale e le predizioni calcola la matrice di confusione. Documentazione disponibile al seguente link [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

**Syntax**:
```python
confusion_matrix(y_true, y_pred)
```

#### `classification_report`

Genera un report testuale che mostra le principali metriche di classificazione. Documentazione disponibile al seguente link [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

**Syntax**:
```python
classification_report(y_true, y_pred)
```

#### `precision_score`

Misura il rapporto tra le istanze positive correttamente previste e il totale delle previsioni positive. Documentazione disponibile al seguente link [precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html).

**Syntax**:
```python
precision_score(y_true, y_pred, average='binary')
```

#### `recall_score`

Misura il rapporto tra le istanze positive correttamente previste e il totale delle istanze positive effettive. Documentazione disponibile al seguente link [recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html)

**Syntax**:
```python
recall_score(y_true, y_pred, average='binary')
```

#### `f1_score`

Calcola la media armonica di precision e recall. Documentazione disponibile al seguente link [f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

**Syntax**:
```python
f1_score(y_true, y_pred, average='binary')
```



### **Guida:**

1. **Calcoliamo la matrice di confusione:** Calcolare la matrice di confusione con `confusion_matrix` e stamparla.

2. **Calcoliamo precision, recall e F1 score:** Calcolare le metriche di valutazione con le funzioni presentate sopra e stamparle.

3. **Calcoliamo il classification report:** Calcolare il classification report e stamparlo.

4. **Stampare la matrice di confusione:** Utilizzare la funzione `plot_confusion_matrix` che vi abbiamo fornito per stampare la matrice di confusione. La funzione ha bisogno di un unico parametro che è la matrice di confusione calcolata al punto 1.

In [ ]:
# Step 1 - Calcolare la matrice di confusione

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, y_predictions_test)

In [ ]:
# Step 2 - Calcolare precision, recall e F1 score

from sklearn.metrics import precision_score, recall_score, f1_score

precision_score_var = precision_score(y_test, y_predictions_test, average='binary')
recall_score_var = recall_score(y_test, y_predictions_test, average='binary')
f1_score_var = f1_score(y_test, y_predictions_test, average='binary')

print(precision_score_var)
print(recall_score_var)
print(f1_score_var)

In [ ]:
# Step 3 - Calcolare il report di classificazione
from sklearn.metrics import classification_report

classification_report_var = classification_report(y_test, y_predictions_test)
print(classification_report_var)

In [ ]:
# Step 4 - Visualizzare la matrice di confusione

def plot_confusion_matrix(cm):
    """
    Visualizza una matrice di confusione come heatmap.
    
    Parameters:
    -----------
    cm : numpy.ndarray
        La matrice di confusione da visualizzare
    """
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
plot_confusion_matrix(cm)

### **ROC Curve, AUC**

Per calcolare la ROC curve, e conseguentemente l' AUC, abbiamo bisogno delle probabilità di predizione. 

Nel caso della **regressione logistica** possiamo utilizzare l' attributo del modello `predict_proba` da utilizzare come segue:

```python
# Estrai le probabilità della classe positiva
y_pred_proba = classifier.predict_proba(X_test_scaled)[:, 1]
```

Per quanto riguarda invece l' **SVM**, è necessario specificare il parametro `probability` = **True** affinchè `predict_proba` funzioni.

```python
# Specifica il parametro probability=True 
classifier = SVC(kernel='linear', C=0.01, probability=True)
```

Una volta estratte le probabilità possiamo utilizzare le funzione di `sklearn.metrics`:

- `roc_curve` 
- `roc_auc_score`

Di seguito è mostrata la sintassi per utilizzare le due funzioni.

---

#### `roc_curve`
```python
# Calcola i valori della curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
```
Calcola i valori della Curva ROC (Receiver Operating Characteristic): Tasso di Falsi Positivi (FPR), Tasso di Veri Positivi (TPR) e soglie.

---

#### `roc_auc_score`
```python
# Calcola AUC
auc = roc_auc_score(y_test, y_pred_proba)
```
Calcola l'Area Under the Curve (AUC) per la ROC, quantificando la capacità del modello di distinguere tra classi positive e negative.

In [ ]:
# Step 1 - Estrarre le probabilità dal modello di regressione logistica
# ATTENZIONE: Per il calcolo della ROC curve ci servono le probabilità della classe positiva.

y_predict_proba_log = model_log.predict_proba(X_test)[:, 1]

In [ ]:
# Step 2 - Allenare l' SVM con il parametro probability=True ed estrarre le probabilità

model_SVM = SVC(kernel='linear', C=0.01, probability=True)
model_SVM.fit(X_train, y_train)
y_predict_proba_SVM = model_SVM.predict_proba(X_test)[:, 1]

In [ ]:
# Step 3 - Calcolare le curve ROC e AUC per entrambe le probabilità (logistic e SVM)

from sklearn.metrics import roc_curve, roc_auc_score

fpr_log, tpr_log, thresholds_log = roc_curve(y_test, y_predict_proba_log)
auc_log = roc_auc_score(y_test, y_predict_proba_log)

fpr_SVM, tpr_SVM, thresholds_SVM = roc_curve(y_test, y_predict_proba_SVM)
auc_SVM = roc_auc_score(y_test, y_predict_proba_SVM)

In [ ]:
# Step 4 - Disegnare le curve ROC per entrambe le probabilità (logistic e SVM)

def plot_roc_curve(fpr, tpr, auc):
    """
    Disegna la curva ROC e stampa il valore AUC.

    Parameters:
    -----------
    fpr : array-like
        Tasso di falsi positivi (False Positive Rate).
    tpr : array-like
        Tasso di veri positivi (True Positive Rate).
    auc : float
        Area sotto la curva (AUC).
    """
    import matplotlib.pyplot as plt

    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {auc:.4f}')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--', lw=2)  # Linea di non discriminazione
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

    # Stampa il valore AUC
    print(f"AUC: {auc:.4f}")

fpr_log, tpr_log, thresholds_log = roc_curve(y_test, y_pred_proba_log)
auc_log = roc_auc_score(y_test, y_pred_proba_log)

fpr_SVM, tpr_SVM, thresholds_SVM = roc_curve(y_test, y_pred_proba_SVM)
auc_SVM = roc_auc_score(y_test, y_pred_proba_SVM)

# **Esercizio 4: Classificazione multi classe**

Se finora abbiamo lavorato soltanto con classificazione binaria, adesso addestriamo nuovamente i classificatori visti sopra, ma nella situazione in cui abbiamo più classi. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle


mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = np.array(mnist.data), np.array(mnist.target.astype(int))

# Utilizziamo soltanto il 20% dei campioni del dataset per questioni di praticità
n_percent = 0.2  

# La funzione train_test_split ci assicura che i dati che rimuoviamo siano bilanciati.
# In questo modo non alteriamo la distribuzione delle classi.
X, _, y, _ = train_test_split(
    X, y, train_size=n_percent, stratify=y, random_state=42
)

X, y = shuffle(X, y)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

In [ ]:
# Step 1 - Dividiamo il dataset in train, validation e test e standardizziamo.
train_fraction = 0.6  
validation_fraction = 0.2  
test_fraction = 0.2  


train_numb = int(train_fraction * X.shape[0])
validation_numb = int(validation_fraction*X.shape[0])

X_train = X[train_numb]
y_train = y[:train_numb]
X_validation = X[train_numb:train_numb+validation_numb]
y_validation = y[train_numb:train_numb+validation_numb]
X_test = X[train_numb+validation_numb:]
y_test = y[train_numb+validation_numb:]

print("Train shape:", X_train.shape)
print("Validation shape:", X_validation.shape)
print("Test shape:", X_test.shape)

#Standardizzazione
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

In [ ]:
# Step 2 - Alleniamo il modello di regressione logistica e calcoliamo le predizioni e prestazioni.
from sklearn.linear_model import LogisticRegression
import numpy as np

# IMPORTANTE:
# Quando istanziamo la classe LogisticRegression utilizziamo come parametri
# max_iter=200 e solver='lbfgs'.

model_log_2 = LogisticRegression(max_iter=1000, solver='lbfgs',C=1.0)
model_log_2.fit(X_train, y_train)
y_predictions_validation = model_log_2.predict(X_validation)
y_predictions_test = model_log_2.predict(X_test)

i = 0
counter = 0
for label in y_predictions_test:
    if label == y_test[i]:
        counter = counter+1
    i = i+1
accuracy_test = counter/len(y_predictions_test)
print("Test Accuracy: ", accuracy_test)

i = 0
counter = 0
for label in y_predictions_validation:
    if label == y_validation[i]:
        counter = counter+1
    i = i+1
accuracy_validation = counter/len(y_predictions_validation)
print("Validation accuracy: ", accuracy_validation)


In [ ]:
# Step 3 - Alleniamo il modello di SVM e calcoliamo le predizioni e prestazioni.
from sklearn.svm import SVC

# IMPORTANTE:
# Quando istanziamo la classe SVC utilizziamo come parametri C=0.01, 
# kernel='linear' e decision_function_shape='ovr'.

from sklearn.svm import SVC
import numpy as np

model_SVM2 = SVC(kernel='linear', C=0.01, decision_function_shape='ovr')
model_SVM2.fit(X_train, y_train)

predictions = model_SVM2.predict(X_test)

i = 0
counter = 0
for label in y_predictions_test:
    if label == y_test[i]:
        counter = counter+1
    i = i+1
accuracy_test = counter/len(y_predictions_test)
print("Test Accuracy: ", accuracy_test)

i = 0
counter = 0
for label in y_predictions_validation:
    if label == y_validation[i]:
        counter = counter+1
    i = i+1
accuracy_validation = counter/len(y_predictions_validation)
print("Validation accuracy: ", accuracy_validation)

# **Esercizio 5: Scrivere una funzione pipeline per l' allenamento di un classificatore**

La funzione `pipeline` prende in input il train set, `X_train` e `y_train`, il validation set, `X_val` e `y_val`, e un dizionario `hyperparams` che contiene una configurazione di training.

La funzione deve:

* Applicare la PCA **se richiesto**.

* Standardizzare i dati **a prescindere che sia richiesto o meno**.

* Allenare un classificatore. Il dizionario avrà una chiave `classifier`, il cui value può essere:

    * `lr` per indicare un modello di **Regressione Logistica**.
    * `svm` per indicare un modello **SVM**. 

* Effettuare le predizioni e utilizzarle per calcolare l' accuracy del classificatore.

In [ ]:
# IMPORTANTE: Eseguire questa cella prima di procedere

def plot_confusion_matrix_multiclass(cm):
    """
    Visualizza una matrice di confusione come heatmap.

    Parameters:
    -----------
    cm : numpy.ndarray
        La matrice di confusione da visualizzare
    """
    # Calcola dinamicamente le etichette delle classi
    class_labels = [f"Class {i}" for i in range(cm.shape[0])]
    
    # Visualizza la heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    
    if hyperparams['use_pca']:
        pca = PCA(n_components=min(X_train.shape[1], X_train.shape[0]))  # Limitare i componenti a minore delle righe/colonne
        X_train = pca.fit_transform(X_train) 
        X_val = pca.transform(X_val)        
        # Implementare codice per la PCA
    
    # Implementare codice per la standardizzazione
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    if hyperparams['classifier'] == 'lr':  
        model = LogisticRegression(max_iter=100, solver='liblinear',C=1.0)
        model.fit(X_train, y_train)
        
    elif hyperparams['classifier'] == 'svm':  
        # Implementare codice per SVM
        model = SVC(kernel='linear', C=0.01, decision_function_shape='ovr')
        model.fit(X_train, y_train)
    # Effettuare predizioni
    y_train_predictions = model.predict(X_train)
    y_validation_predictions = model.predict(X_validation)
    
    i = 0
    counter = 0
    for label in y_predictions_test:
        if label == y_test[i]:
            counter = counter+1
        i = i+1
    accuracy_test = counter/len(y_predictions_test)
    print("Test Accuracy: ", accuracy_test)
    
    i = 0
    counter = 0
    for label in y_predictions_validation:
        if label == y_validation[i]:
            counter = counter+1
          i = i+1
    accuracy_validation = counter/len(y_predictions_validation)
    print("Validation accuracy: ", accuracy_validation)

    
    # Calcolare la matrice di confusione
    cm = confusion_matrix(y_test, y_predictions_test)
    # Visualizzare la matrice di confusione
    plot_confusion_matrix_multiclass(cm)

In [ ]:
# Provare la funzione pipeline su tutte le configurazioni presenti qui di seguito

hyperparams_1 = {
    'use_pca': False,  
    'classifier': 'svm',  
}

hyperparams_2 = {
    'use_pca': True,  
    'classifier': 'svm',  
}

hyperparams_3 = {
    'use_pca': False,  
    'classifier': 'lr',  
}

hyperparams_4 = {
    'use_pca': True,  
    'classifier': 'lr',  
}

pipeline(X_train, y_train, X_validation, y_validation, hyperparams_1)
pipeline(X_train, y_train, X_validation, y_validation, hyperparams_2)
pipeline(X_train, y_train, X_validation, y_validation, hyperparams_3)
pipeline(X_train, y_train, X_validation, y_validation, hyperparams_4)